# Solution Lab 13 - Winde Data Clustering with K-means
## Pyspark Kmeans

### Team Members:


*   Marian Sedano
*   Luis Fontes
* Gabriel Olvera



## Pyspark Environment

In [1]:
#Pyspark
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# Check this site for the latest download link
# https://www.apache.org/dyn/closer.lua/spark
!wget -q https://dlcdn.apache.org/spark/spark-3.5.2/spark-3.5.2-bin-hadoop3.tgz
!tar xf spark-3.5.2-bin-hadoop3.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,466 kB]
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,424 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Package

## Solution - Kmeans and Silhouette

In [6]:
#Import
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.sql import SparkSession

#Initialize
spark = SparkSession.builder.appName("WineClustering").getOrCreate()

In [3]:
!unzip wines.zip -d wines_ds

Archive:  wines.zip
  inflating: wines_ds/wine-clustering.csv  


In [7]:
#Dataset load
file_path = "/content/wines_ds/wine-clustering.csv"
df = spark.read.csv(file_path, header=True, inferSchema=True)

# Drop Null
df = df.dropna()

In [8]:
# Vector Assembler
from pyspark.ml.feature import VectorAssembler

feature_columns = df.columns
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
df = assembler.transform(df)

# KMeans Values
k_values = [2, 10, 15, 20]
results = {}

In [9]:
for k in k_values: #Value 2,10,15,20
    kmeans = KMeans().setK(k).setSeed(1).setFeaturesCol("features")
    model = kmeans.fit(df)

    #Predict
    predictions = model.transform(df)

    #Evaluete and obtain silhoutte
    evaluator = ClusteringEvaluator()
    silhouette_score = evaluator.evaluate(predictions)
    results[k] = silhouette_score

    print(f"For k = {k}, Silhouette Score = {silhouette_score}")

For k = 2, Silhouette Score = 0.8213603513331723
For k = 10, Silhouette Score = 0.6810238482437947
For k = 15, Silhouette Score = 0.6844672306874956
For k = 20, Silhouette Score = 0.6393344257672275


In [10]:
# Stop Spark
spark.stop()